In [1]:
#! /usr/bin/env python3
# https://github.com/huggingface/transformers/blob/main/examples/research_projects/rag-end2end-retriever/finetune_rag.py
import numpy as np
from sys import getsizeof
import os.path
from transformers import T5Tokenizer
from icecream import ic
import time
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast
import pandas as pd
import random
import copy
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers

In [2]:
class DPR(pl.LightningModule):
    """
    Implementation of the DPR module :
    Encode all documents (contexts), and query with different BERT encoders.
    Similarity measure with dot product.
    """

    def __init__(self,
                 query_model_name: str,
                 context_model_name: str,
                 train_batch_size=2,
                 val_batch_size=2,
                 learning_rate=1e-5):
        super().__init__()
        self.query_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained(query_model_name)
        self.context_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained(context_model_name)
        self.query_model = DPRQuestionEncoder.from_pretrained(query_model_name)
        self.context_model = DPRContextEncoder.from_pretrained(context_model_name)

        self.learning_rate = learning_rate
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size

        self.train_ds, self.val_ds, self.test_ds = None, None, None

        # self.loss_fn = nn.NLLLoss()
        self.loss_fn = self.get_loss_fn()

    def get_sets(self, train_val_test):
        self.train_ds, self.val_ds, self.test_ds = train_val_test

    def encode_queries(self, queries):
        qry_enc = lambda qry: self.query_tokenizer(qry, truncation=True,
                                                   max_length=512,
                                                   padding="max_length",
                                                   return_tensors="pt")

        queries_encoding = pd.DataFrame(queries["text"].apply(qry_enc).tolist())
        queries["input_ids"] = queries_encoding["input_ids"]
        queries["attention_mask"] = queries_encoding["attention_mask"]
        return queries

    def encode_contexts(self, contexts: pd.DataFrame):
        ctx_enc = lambda ctx: self.context_tokenizer(ctx, truncation=True,
                                                     max_length=512,
                                                     padding="max_length",
                                                     return_tensors='pt')
        contexts_encoding = pd.DataFrame(contexts["text"].apply(ctx_enc).tolist())
        contexts["input_ids"] = contexts_encoding["input_ids"]
        contexts["attention_mask"] = contexts_encoding["attention_mask"]
        return contexts

    def decode_contexts(self, contexts_encodings: list):
        contexts = [self.context_tokenizer.decode(c) for c in contexts_encodings]
        return contexts

    def get_dense_query(self, query):
        dense_query = self.query_model(input_ids=query["input_ids"],
                                       attention_mask=query["attention_mask"])["pooler_output"]
        return dense_query

    def get_dense_contexts(self, contexts):
        dense_embeddings = []
        for context in contexts:
            embedding = self.context_model(input_ids=context["input_ids"],
                                           attention_mask=context["attention_mask"])
            dense_embeddings.append(embedding["pooler_output"])
        return torch.cat(dense_embeddings)

    def dot_product(self, query, contexts):
        sim = query.squeeze().matmul(contexts.T)
        return sim.squeeze()

    def train_dataloader(self):
        return DataLoader(copy.deepcopy(self.train_ds),
                          batch_size=self.train_batch_size,
                          num_workers=4,
                          shuffle=True,
                          collate_fn=lambda x: x)

    def val_dataloader(self):
        return DataLoader(copy.deepcopy(self.val_ds),
                          batch_size=self.val_batch_size,
                          num_workers=4,
                          shuffle=True,
                          collate_fn=lambda x: x)

    def test_dataloader(self):
        return DataLoader(copy.deepcopy(self.test_ds),
                          batch_size=self.val_batch_size,
                          num_workers=4,
                          shuffle=True,
                          collate_fn=lambda x: x)

    def get_loss_fn(self):
        """negative log likelihood from DPR paper."""

        def loss_fn(similarity):
            return -torch.log(similarity[0].exp() / similarity.exp().sum())

        return loss_fn

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),
                                     lr=self.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        """
        :param query:
        :param return_contexts:
        :param k:
        :return:
        """
        self.context_model.train()
        self.query_model.train()
        loss = 0
        for data in batch :
            query_dense = self.get_dense_query(data["query"])
            contexts_dense = self.get_dense_contexts([data["positive"], *data["negative"]])
            similarity = self.dot_product(query_dense, contexts_dense)
            loss += self.loss_fn(similarity)

        loss /= len(batch)
        self.log('train/loss_step', loss.item(), on_step=True)
        self.log('train/loss_epoch', loss.item(), on_step=False, on_epoch=True)

        return loss

    def training_epoch_end(self, outputs):
        print('Finishing epoch ', self.current_epoch)

    def validation_step(self, batch, batch_idx):
        self.context_model.eval()
        self.query_model.eval()
        loss = 0
        with torch.no_grad():
            for data in batch:
                self.context_model.train()
                self.query_model.train()
                query_dense = self.get_dense_query(data["query"])
                contexts_dense = self.get_dense_contexts([data["positive"], *data["negative"]])
                similarity = self.dot_product(query_dense, contexts_dense)
                loss += self.loss_fn(similarity)

            loss /= len(batch)
            self.log('Val/loss_step', loss.item(), on_step=True)
            self.log('Val/loss_epoch', loss.item(), on_step=False, on_epoch=True)

    def validation_epoch_end(self, outputs):
        print('Validation epoch ', self.current_epoch)

In [3]:
dpr = DPR(context_model_name="facebook/dpr-ctx_encoder-single-nq-base",
          query_model_name="facebook/dpr-question_encoder-single-nq-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequ

In [4]:
class CorpusDataset(pd.DataFrame):
    def __init__(self, path_to_file: str, context_encoder):
        super().__init__(self.get_df(path_to_file, context_encoder))

    def get_df(self, path, context_encoder):
        df = pd.read_csv(path)
        df = df.loc[df["text"] != "\n"]
        df = context_encoder(df)
        return df


class QueryDataset(Dataset):
    def __init__(self, path_to_file: str, text_column: str, query_encoder, corpus, nb_irrelevant=1):
        self.df = self.get_df(path_to_file, text_column, query_encoder)
        self.corpus = corpus
        self.nb_irrelevant = nb_irrelevant
        self.count_doc = [0 for _ in range(len(self.df))]

    def get_df(self, path, text_column=None, query_encoder=None):
        def parse_ids(ids_str):
            ids_list = [id[1:-1] for id in ids_str[1:-1].split(", ")]
            return ids_list

        df = pd.read_csv(path)
        columns = ["query",
                   "outline",
                   "text",
                   "paragraphs_id"]
        df = df[["query",
                 "outline",
                 "text_" + text_column,
                 "paragraphs_id"]]
        df = df.loc[df["outline"].map(len) > 0]
        df["text"] = df["text_" + text_column]
        df["paragraphs_id"] = df["paragraphs_id"].apply(parse_ids)
        df = df.loc[df["text"] != "\n"]
        df = df[columns]
        df = query_encoder(df)
        return df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, item):
        row = self.df.iloc[item]
        docs_id = row["paragraphs_id"]
        query = {"query": row["query"],
                 "input_ids": row["input_ids"],
                 "attention_mask": row["attention_mask"]}
        positive = self.get_positive(docs_id[self.count_doc[item]])
        negative = self.get_negative(docs_id)
        element = {"query": query,
                   "positive": positive,
                   "negative": negative}
        self.count_doc[item] = (self.count_doc[item] + 1) % len(docs_id)
        return element

    def get_positive(self, doc_id):
        positive = {"doc_id": doc_id}
        row = self.corpus[self.corpus["id"] == doc_id]
        positive["input_ids"] = row["input_ids"].values[0]
        positive["attention_mask"] = row["attention_mask"].values[0]
        return positive

    def get_negative(self, docs_id_positive):
        negatives_documents = []
        for _ in range(self.nb_irrelevant):
            item_random = np.random.randint(0, len(self.corpus))
            row_negative = self.corpus.iloc[item_random]
            while row_negative["id"] in docs_id_positive:
                item_random = np.random.randint(0, len(self.corpus))
                row_negative = self.corpus.iloc[item_random]
            negative = {"doc_id": row_negative["id"],
                        "input_ids": row_negative["input_ids"],
                        "attention_mask": row_negative["attention_mask"]}
            negatives_documents.append(negative)
        return negatives_documents

In [5]:
corpus_train = CorpusDataset(path_to_file="../../../data-subset_pre_processed/fold-1/corpus_train.csv",
                             context_encoder=dpr.encode_contexts)
corpus_val = CorpusDataset(path_to_file="../../../data-subset_pre_processed/fold-2/corpus_train.csv",
                           context_encoder=dpr.encode_contexts)
corpus_test = CorpusDataset(path_to_file="../../../data-subset_pre_processed/fold-3/corpus_train.csv",
                            context_encoder=dpr.encode_contexts)

In [6]:
text_column = "w/o_heading_first_sentence_by_paragraph"

ds_train = QueryDataset(path_to_file="../../../data-subset_pre_processed/fold-1/articles_train_all_ids.csv",
                        text_column=text_column,
                        query_encoder=dpr.encode_queries,
                        corpus=corpus_train,
                        nb_irrelevant=2)
# train_dataloader = DataLoader(ds_train, batch_size=2, shuffle=True)

ds_val = QueryDataset(path_to_file="../../../data-subset_pre_processed/fold-2/articles_train_all_ids.csv",
                      text_column=text_column,
                      query_encoder=dpr.encode_queries,
                      corpus=corpus_val)
# val_dataloader = DataLoader(ds_val, batch_size=2, shuffle=True)

ds_test = QueryDataset(path_to_file="../../../data-subset_pre_processed/fold-3/articles_train_all_ids.csv",
                       text_column=text_column,
                       query_encoder=dpr.encode_queries,
                       corpus=corpus_test)
# test_dataloader = DataLoader(ds_test, batch_size=2, shuffle=True)

In [7]:
dpr.get_sets(train_val_test=(ds_train, ds_val, ds_test))

In [8]:
logger = pl_loggers.TensorBoardLogger("./checkpoints", name="dpr_retriever")
checkpoint_callback = ModelCheckpoint(monitor="Val/loss_epoch", mode="min", save_top_k=2, every_n_epochs=2)

In [9]:
trainer = Trainer(logger=logger,
                  precision=32,
                  gpus=-1,
                  strategy='dp',
                  max_epochs=100,
                  callbacks=[checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model=dpr)

Missing logger folder: ./checkpoints/dpr_retriever
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | query_model   | DPRQuestionEncoder | 108 M 
1 | context_model | DPRContextEncoder  | 108 M 
-----------------------------------------------------
217 M     Trainable params
0         Non-trainable params
217 M     Total params
871.133   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/legmint/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/legmint/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Validation epoch  0


/home/legmint/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 5.79 GiB total capacity; 4.46 GiB already allocated; 13.25 MiB free; 4.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()


In [ ]:
# offline
contexts_ids = DPRContextEncoderTokenizerFast(contexts)  # ids of vocabulary in Encoder's dictionary
queries_ids = DPRQuestionEncoderTokenizerFast(queries)  # ids of vocabulary in Encoder's dictionary

corpus = {
    "context_idex_1": context_ids_1,
    "context_idex_2": context_ids_2,
    "context_idex_3": context_ids_3,
    "context_idex_4": context_ids_4,
    ...
}

batches = [
    {"queries_ids": queries_ids,
     "contexts_indexes": contexts_idexes},  # ground-truth of the documents indexes in corpus
    {...},
    ...
]

# online
for epoch in epochs:
    contexts_embedding = DPRContextEncoder(contexts_ids)

    for batch in batches:
        queries_embedding = DPRQuestionEncoder(batch["queries_ids"])
        top_k = get_to_k(contexts_embedding, queries_embedding)
        loss = CrossEntropyLoss(top_k, batch["contexts_indexes"])
        loss.backward()
